In [ ]:
# Load data preprocessing libs
import pandas as pd   # dataset
import numpy as np  # maths
from datetime import datetime,date # date time lib

import re
from bs4 import BeautifulSoup  # removing html tags

# Load vectorizer and similarity measure
from sklearn.feature_extraction.text import TfidfVectorizer  # term freq inverse doc. freq.
from sklearn.metrics.pairwise import cosine_similarity  # measuring the similarity of text

import tkinter   # User interface
import threading
from tkinter import *
from tkinter import messagebox
import speech_recognition as speech # taking input through voice
import import_ipynb
# from mailing import mail

In [ ]:
# importing the question and answer dataset in separate dataframe.
q = pd.read_csv("Dataset/Questions.csv",encoding='latin-1')
an = pd.read_csv("Dataset/AnswersV2.csv",encoding='latin-1')

In [ ]:
# No. of question dataset
print(len(q.index))

In [ ]:
# first five instances of answer.
an.head()

In [ ]:
an.rename(columns={'ParentId':'QId'},inplace=True)
an.head()

In [ ]:
q.rename(columns={'Id':'QId'},inplace=True)
an.rename(columns={'Id':'AId'},inplace=True)

In [ ]:
an = an[an['Score']>5]
an.head()

In [ ]:
q.head()

In [ ]:
# merging the dataframe on question id.
df = q.merge(an, on='QId')
df.head()

In [ ]:
# dropping the irrelevant columns
df.drop(columns=['OwnerUserId_x','CreationDate_x','Score_x','OwnerUserId_y','CreationDate_y'],inplace=True)
df.head()

In [ ]:
# renaming columns
df.rename(columns={'Body_x':'Question','Body_y':'Answer','Score_y':'Score'},inplace=True)

In [ ]:
df.head()

In [ ]:
# Removing the HTML tags from the Body(Text) to process and give the answer effectively.

df['Answer'] = df['Answer'].apply(lambda x:BeautifulSoup(x).get_text())
df['Question'] = df['Question'].apply(lambda x:BeautifulSoup(x).get_text())

In [ ]:
df.head()

In [ ]:
# Vectorizing the dataset.

vectorizer = TfidfVectorizer()
vectorizer.fit(np.concatenate((df.Question, df.Answer)))

In [ ]:
Question_vectors = vectorizer.transform(df.Question)

In [ ]:
# This threshold represents the no of days after which answer score is updated.

threshold = 30
answer = []

In [ ]:
# the method to find the answer

def getAnswerWithHighestScore(answers):
    r = np.argmax(answers['latest_score']-answers['alternate'])
    if (datetime.strptime(str(date.today()),"%Y-%m-%d")-datetime.strptime(answers['date'].iloc[0],"%Y-%m-%d")).days > threshold:
        df[df['QId'] == answers['QId'].iloc[0]]['date'] = date.today()
        df[df['QId'] == answers['QId'].iloc[0]]['alternate']=df[df['QId'] == answers['QId'].iloc[0]]['latest_score']
        an[an['QId'] == answers['QId'].iloc[0]]['date'] = date.today()
        an[an['QId'] == answers['QId'].iloc[0]]['alternate']=df[df['QId'] == answers['QId'].iloc[0]]['latest_score']
        an.to_csv("Dataset/AnswersV2.csv",index=False)
    return [answers.iloc[r]['Answer'],answers.iloc[r]['AId'],answers.iloc[r]['alternate']]

In [ ]:
def get_answer(row):
    global answer
    
    qid = df.iloc[row][0]  # question id similar to the question asked by user
    answers = df.loc[df['QId'] == qid]
    answer = getAnswerWithHighestScore(answers)  # retreive the ans with highest score 
    return [answer[0],answer[2]]

In [ ]:
def chatbot_response(msg):
    # removing HTML tags from the response
    input_question =BeautifulSoup(msg).get_text()

    # Locate the closest question
    input_question_vector = vectorizer.transform([input_question])

    # Compute similarities
    similarities = cosine_similarity(input_question_vector, Question_vectors)

    # Find the closest question
    closest = np.argmax(similarities, axis=1)
    a =  get_answer(closest[0])
    return [a[0],a[1]]
#     return df.Answer.iloc[closest].values[0]

In [ ]:
# defining upvote and downvote button variables

global check,upvoted,downvoted
check = True
upvoted = False
downvoted = False

In [ ]:
def upvote():
    global upvoted,downvoted
    if not upvoted:
        upvoted = True
        downvoted = False
        r1 = df.index[df['AId'] == answer[1]]
        r2 = an.index[df['AId'] == answer[1]]
        score = int(df.iloc[r1]['latest_score'].values[0])
        score+=1
        updateScore(score)
        df.loc[r1,'latest_score'] = score
        an.loc[r2,'latest_score'] = score
        an.to_csv("Dataset/AnswersV2.csv",index=False)
        messagebox.showinfo("UPVOTE!!!","ANSWER IS UPVOTED")

In [ ]:
def downvote():
    global upvoted,downvoted
    if not downvoted:
        downvoted = True
        upvoted = False
        r1 = df.index[df['AId'] == answer[1]]
        r2 = an.index[df['AId'] == answer[1]]
        score = int(df.iloc[r1]['latest_score'].values[0])
        score-=1
        updateScore(score)
#         if(score <0):
#             mail(answer[1])
        df.loc[r1,'latest_score'] = score
        an.loc[r2,'latest_score'] = score
        an.to_csv("Dataset/AnswersV2.csv",index=False)
        messagebox.showinfo("DOWNVOTE!!!","ANSWER IS DOWNVOTED")

In [ ]:
def updateScore(newScore):
    score_text.delete("1.0",END)
    score_text.insert("1.0",newScore)

In [ ]:
class TrieNode():
    def __init__(self):
        # Initialising one node for trie
        self.children = {}
        self.last = False
        self.answers=[]


class Trie():
    def __init__(self):

        # Initialising the trie structure.
        self.root = TrieNode()
        self.answers = []

    def insert(self, key):

        # Inserts a key into trie if it does not exist already.
        # And if the key is a prefix of the trie node, just
        # marks it as leaf node.
        node = self.root

        for a in key:
            if not node.children.get(a):
                node.children[a] = TrieNode()

            node = node.children[a]

        node.last = True

    def suggestionsRec(self, node, word):

        # Method to recursively traverse the trie
        # and return a whole word.
        if node.last:
            self.answers.append(word)

        for a, n in node.children.items():
            self.suggestionsRec(n, word + a)

    def printAutoSuggestions(self, key):

        # Returns all the words in the trie whose common
        # prefix is the given key thus listing out all
        # the suggestions for autocomplete.
        node = self.root

        for a in key:
            # no string in the Trie has this prefix
            if not node.children.get(a):
                return 0
            node = node.children[a]

        # If prefix is present as a word, but
        # there is no subtree below the last
        # matching node.
        if not node.children:
            return -1

        self.suggestionsRec(node, key)
        return 1

    def getAnswers(self):
        return self.answers

    def reset(self):
        self.answers =[]

In [ ]:
que= q.Title.tolist()

In [ ]:
ob1 = Trie()
for i in range(30000):
    ob1.insert(que[i])

In [ ]:
from pynput.keyboard import Key, Listener

In [ ]:
anss = []

In [ ]:
import functools

In [ ]:
def autoComplete(text=''):
    global anss
    ob1.printAutoSuggestions(text)
    anss = ob1.getAnswers()
    ob1.reset()


In [ ]:
def speechToText():
    obj = speech.Recognizer()
    print("PyBot is listening you...")
    with speech.Microphone() as microphone:
        try:
            print("Listening ...")
            audio = obj.listen(microphone)
            print("Processing the Voice ...")
            query = obj.recognize_google(audio,language = 'eng-in')
            print("Recognized the Question ...")
            print(query)
            EntryBox.delete('1.0',END)
            EntryBox.insert('1.0', query)
            send()
        except Exception as e:
            print(e)


def send():
    global upvoted,downvoted
    upvoted = False
    downvoted = False
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res[0] + '\n\n')
        updateScore(res[1])
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)



base = Tk()
base.title("PyBOT")
base.geometry("600x650")
base.resizable(width=FALSE, height=FALSE)
# base.withdraw()
# def popup(vote):
#     messagebox.showinfo("Alert","Answer was "+vote)

def sp():
    thread = threading.Thread(target=speechToText())
    thread.start()

score_text = Text(base,bg="white")
upvote = Button(base, text="UPVOTE", fg="black",bg="white",command=upvote)
upvote.place(x=30, y=300, height=40, width=90)

downvote = Button(base, text="DOWNVOTE", fg="black",bg="white",command=downvote)
downvote.place(x=180, y=300, height=40, width=90)

speechButton = Button(base, text="Mic", fg="black",bg="white",command=sp)
speechButton.place(x=350, y=300, height=40, width=50)
score_text.place(x=500,y=300, height=40,width=40)

ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial")
ChatLog.config(state=DISABLED)

scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")

ListBox = Listbox(base, bg="white",width="29", height="5", font="Arial")

scrollbar.place(x=575,y=6, height=280)
ChatLog.place(x=6,y=6, height=280, width=800)
EntryBox.place(x=128, y=380, height=120, width=500)
SendButton.place(x=6, y=380, height=120)
ListBox.place(x=6, y=520, height=100,width = 600)

def update(data):
	# Clear the listbox
	ListBox.delete(0, END)

	for item in data:
		ListBox.insert(END, item)

# Update entry box with listbox clicked
def populate(e):
	# Delete whatever is in the entry box
	EntryBox.delete("1.0", END)

	# Add clicked list item to entry box
	EntryBox.insert("1.0", ListBox.get(ANCHOR))

def getCompleted(event):
    msg = EntryBox.get("1.0",'end-1c').strip()

    autoComplete(msg)
    update(anss[:5])

ListBox.bind("<<ListboxSelect>>", populate)
base.bind("<Key>",getCompleted)
base.mainloop()